In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

c:\users\hp\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()
test_df = pd.read_csv('test.csv')
test_df.shape

(418, 11)

In [3]:
#sns.countplot()
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
test_df.pop("Cabin")
test_df['Age']=test_df['Age'].fillna(test_df['Age'].dropna().mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].dropna().mean())
em = pd.get_dummies(test_df["Embarked"],drop_first=True)
sx = pd.get_dummies(test_df["Sex"],drop_first=True)
pc = pd.get_dummies(test_df["Pclass"],drop_first=True)
test_dff = pd.concat([test_df,em,sx,pc],axis=1)
test_dff.drop(['PassengerId',"Pclass","Name","Sex","Embarked","Ticket"],axis=1,inplace=True)

In [5]:
test_dff

,Age,SibSp,Parch,Fare,Q,S,male,2,3
0,34.50000,0,0,7.8292,1,0,1,0,1
1,47.00000,1,0,7.0000,0,1,0,0,1
2,62.00000,0,0,9.6875,1,0,1,1,0
3,27.00000,0,0,8.6625,0,1,1,0,1
4,22.00000,1,1,12.2875,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
413,30.27259,0,0,8.0500,0,1,1,0,1
414,39.00000,0,0,108.9000,0,0,0,0,0
415,38.50000,0,0,7.2500,0,1,1,0,1
416,30.27259,0,0,8.0500,0,1,1,0,1


In [6]:
train_df.dropna(inplace=True)
train_df.pop("Cabin")
em = pd.get_dummies(train_df["Embarked"],drop_first=True)
sx = pd.get_dummies(train_df["Sex"],drop_first=True)
pc = pd.get_dummies(train_df["Pclass"],drop_first=True)
train_df = pd.concat([train_df,em,sx,pc],axis=1)
train_df.drop(["PassengerId","Pclass","Name","Sex","Embarked","Ticket"],axis=1,inplace=True)
x = train_df.drop("Survived",1)
y = train_df.Survived.values

In [7]:
x.shape

(183, 9)

In [8]:
std = StandardScaler()
std.fit(x.values)
x_std = std.transform(x.values)
std.fit(test_dff.values)
x_test_data = std.transform(test_dff.values)
x_train,x_test,y_train,y_test = train_test_split(x_std,y,test_size=0.1,random_state=42)
x_test_data.shape

(418, 9)

In [9]:
def Print_result(clf,x_train,x_test,y_train,y_test,train=True):
    """
    print the classification report, confusion matrix and accuracy score
    """
    if train:
        """
        Training Result
        """
        print("Train Result...\n")
        print("Accuracy:\n{0:.4f}".format(accuracy_score(y_train,clf.predict(x_train))))
        print("classification report:\n{}".format(classification_report(y_train,clf.predict(x_train))))
        print("confusion matrix:\n{}".format(confusion_matrix(y_train,clf.predict(x_train))))
        
        cross = cross_val_score(clf,x_train,y_train,cv=15,scoring="accuracy")
        print("Average_score:\n{0:.4f}".format(cross.mean()))
        print("Accuracy_std:\n{0:.4f}".format(cross.std()))
        
    if train==False:
        """
        Testing Result
        """
        print("Test Result...\n")
        print("Accuracy:\n{0:.4f}".format(accuracy_score(y_test,clf.predict(x_test))))
        print("classification report:\n{}".format(classification_report(y_test,clf.predict(x_test))))
        print("confusion matrix:\n{}".format(confusion_matrix(y_test,clf.predict(x_test))))
        
        cross = cross_val_score(clf,x_test,y_test,cv=10,scoring="accuracy")
        print("Average_score:\n{0:.4f}".format(cross.mean()))
        print("Accuracy_std:\n{0:.4f}".format(cross.std()))

In [10]:
log = LogisticRegression(penalty="l2")
log.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
Print_result(log,x_train,x_test,y_train,y_test,train=True)
Print_result(log,x_train,x_test,y_train,y_test,train=False)

Train Result...

Accuracy:
0.7988
classification report:
              precision    recall  f1-score   support

           0       0.69      0.65      0.67        51
           1       0.84      0.87      0.86       113

    accuracy                           0.80       164
   macro avg       0.77      0.76      0.76       164
weighted avg       0.80      0.80      0.80       164

confusion matrix:
[[33 18]
 [15 98]]
Average_score:
0.7558
Accuracy_std:
0.1401
Test Result...

Accuracy:
0.6842
classification report:
              precision    recall  f1-score   support

           0       0.71      0.56      0.63         9
           1       0.67      0.80      0.73        10

    accuracy                           0.68        19
   macro avg       0.69      0.68      0.68        19
weighted avg       0.69      0.68      0.68        19

confusion matrix:
[[5 4]
 [2 8]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Average_score:
0.5500
Accuracy_std:
0.3500


In [12]:
#output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': y_pred})
#output.to_csv('submission2.csv', index=False)
#output

In [13]:
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [14]:
paras = {"leaf_size":[20,30,10,40,50],
         "p":[2,3,4,5],
         "n_neighbors":[5,6,7,8,9,10]
        }
grid_knn = GridSearchCV(knn,paras,n_jobs=-1,verbose=1,cv=5)
grid_knn.fit(x_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'leaf_size': [20, 30, 10, 40, 50],
                         'n_neighbors': [5, 6, 7, 8, 9, 10],
                         'p': [2, 3, 4, 5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [15]:
y_pred = grid_knn.predict(x_test_data)

In [16]:
Print_result(knn,x_train,x_test,y_train,y_test,train=True)
Print_result(knn,x_train,x_test,y_train,y_test,train=False)

Train Result...

Accuracy:
0.8293
classification report:
              precision    recall  f1-score   support

           0       0.76      0.67      0.71        51
           1       0.86      0.90      0.88       113

    accuracy                           0.83       164
   macro avg       0.81      0.78      0.79       164
weighted avg       0.83      0.83      0.83       164

confusion matrix:
[[ 34  17]
 [ 11 102]]
Average_score:
0.7370
Accuracy_std:
0.1302
Test Result...

Accuracy:
0.6316
classification report:
              precision    recall  f1-score   support

           0       0.62      0.56      0.59         9
           1       0.64      0.70      0.67        10

    accuracy                           0.63        19
   macro avg       0.63      0.63      0.63        19
weighted avg       0.63      0.63      0.63        19

confusion matrix:
[[5 4]
 [3 7]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Average_score:
0.6000
Accuracy_std:
0.3742


In [17]:
grid_knn.best_score_

0.7918560606060606

In [18]:
output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': y_pred})
output.to_csv('submission4.csv', index=False)
output.shape

(418, 2)

In [19]:
de_clf = xgb.XGBClassifier(max_depth=3,learning_rate=0.0001,n_jobs=-1,
                           gamma=1,n_estimators=500)
de_clf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.0001, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
Print_result(de_clf,x_train,x_test,y_train,y_test,train=True)
Print_result(de_clf,x_train,x_test,y_train,y_test,train=False)

Train Result...

Accuracy:
0.7927
classification report:
              precision    recall  f1-score   support

           0       0.61      0.94      0.74        51
           1       0.96      0.73      0.83       113

    accuracy                           0.79       164
   macro avg       0.79      0.83      0.78       164
weighted avg       0.85      0.79      0.80       164

confusion matrix:
[[48  3]
 [31 82]]
Average_score:
0.7248
Accuracy_std:
0.1263
Test Result...

Accuracy:
0.7368
classification report:
              precision    recall  f1-score   support

           0       0.67      0.89      0.76         9
           1       0.86      0.60      0.71        10

    accuracy                           0.74        19
   macro avg       0.76      0.74      0.73        19
weighted avg       0.77      0.74      0.73        19

confusion matrix:
[[8 1]
 [4 6]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Average_score:
0.4500
Accuracy_std:
0.4153


In [21]:
rd_clf = RandomForestClassifier(n_estimators=200,
                            criterion='gini',
                            max_depth=3)
boost_clf = AdaBoostClassifier(base_estimator=rd_clf,n_estimators=50,
                            learning_rate=1.0)

paras = {"n_estimators":[50,500,100],
        "learning_rate":[0.1,1.0,0.01]}

grid_rd = GridSearchCV(boost_clf,paras,n_jobs=-1,verbose=1,cv=5)
grid_rd.fit(x_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 113.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=RandomForestClassifier(bootstrap=True,
                                                                                ccp_alpha=0.0,
                                                                                class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=3,
                                                                                max_features='auto',
                                                                                max_leaf_nodes=None,
                                                                                max_samples=None,
                                                                                min_impurity_decrease=0.0,
             

In [ ]:
Print_result(grid_rd,x_train,x_test,y_train,y_test,train=True)
Print_result(grid_rd,x_train,x_test,y_train,y_test,train=False)

Train Result...

Accuracy:
0.8415
classification report:
              precision    recall  f1-score   support

           0       0.82      0.63      0.71        51
           1       0.85      0.94      0.89       113

    accuracy                           0.84       164
   macro avg       0.83      0.78      0.80       164
weighted avg       0.84      0.84      0.83       164

confusion matrix:
[[ 32  19]
 [  7 106]]
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 56.4min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 630.3min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 40.4min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
ch 

In [ ]:
clf = DecisionTreeClassifier(criterion='gini',splitter='best',
                        max_depth=2,min_samples_split=2,
                        min_samples_leaf=5)

paras = {"max_depth":[2,3,4,5],
            "min_samples_split":[2,3,4,5],
            "min_samples_leaf":[5,6,3,4]}

grid_clf = GridSearchCV(clf,paras,n_jobs=-1,verbose=1,cv=5)
grid_clf.fit(x_train,y_train)

In [ ]:
Print_result(grid_clf,x_train,x_test,y_train,y_test,train=True)
Print_result(grid_clf,x_train,x_test,y_train,y_test,train=False)

In [ ]:
clf = svm.SVC(kernel="rbf",
              C=1.0,gamma=0.1)

paras = {"kernel":["linear","hinge","rbf"],
            "C":[1.0,0.1,1.00],
            "gamma":[0.1,0.11,0.01,1.0,"scale"],
             "degree":[2,3],
            "shrinking":[True,False],
            "probability":[True,False]
        }
grid_svm = GridSearchCV(clf,paras,n_jobs=-1,verbose=1,cv=5)
grid_svm.fit(x_train,y_train)

In [ ]:
Print_result(grid_svm,x_train,x_test,y_train,y_test,train=True)
Print_result(grid_svm,x_train,x_test,y_train,y_test,train=False)

In [ ]:
clf = SGDClassifier(random_state=3,loss='modified_huber',penalty='l2',alpha=0.001,
                    l1_ratio=0.15, fit_intercept=True,max_iter=200,
                    tol=0.001,epsilon=0.011,n_jobs=-1,
                    learning_rate='optimal',eta0=0.01,power_t=0.15,
                    early_stopping=False,validation_fraction=0.001,
                    n_iter_no_change=1,warm_start=True)
boost_clf = AdaBoostClassifier(base_estimator=clf,n_estimators=50,
                            learning_rate=1.0)

paras = {"n_estimators":[50,500,100],
        "learning_rate":[0.1,1.0,0.01]}

grid_boost = GridSearchCV(boost_clf,paras,n_jobs=-1,verbose=1,cv=5)
grid_boost.fit(x_train,y_train)

In [ ]:
Print_result(grid_boost,x_train,x_test,y_train,y_test,train=True)
Print_result(grid_boost,x_train,x_test,y_train,y_test,train=False)